In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

tf.__version__

'2.7.0'

# Optional GPU Setup

In [5]:
######## GPU CONFIGS FOR RTX 2070 ###############
## Please ignore if not training on GPU       ##
## this is important for running CuDNN on GPU ##

tf.keras.backend.clear_session() #- for easy reset of notebook state

# chck if GPU can be seen by TF
tf.config.list_physical_devices('GPU')
# only if you want to see how commands are executed
#tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  print(gpus[0])
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
###############################################

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
1 Physical GPUs, 1 Logical GPU


# Loading IMDB Data

In [6]:
# using TFDS dataset
# note: as_supervised converts dicts to tuples
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", 
                                with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test", 
                      as_supervised=True)

In [4]:
# Check and example from the dataset
for example, label in imdb_train.take(1):
    print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


# Create Vocab and Encode

In [7]:
# Use the default tokenizer settings
tokenizer =  tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
MAX_TOKENS = 0

for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)

In [8]:
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set,
                                                   lowercase=True,
                                                   tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size

print(vocab_size, MAX_TOKENS)

93931 2525


In [7]:
# Lets verify tokenization and encoding works
for example, label in imdb_train.take(1):
    print(example)
    encoded = imdb_encoder.encode(example.numpy())
    print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
this was an absolutely terrible movie don t be lured in by christopher walken or michael ironside both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie s ridiculous storyline this 

In [9]:
# transformation functions to be used with the dataset
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post', 
                                 maxlen=150)
    return np.array(pad[0], dtype=np.int64)  


def encode_tf_fn(sample, label):
    encoded = tf.py_function(encode_pad_transform, 
                                       inp=[sample], 
                                       Tout=(tf.int64))
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label

In [10]:
# test the transformation on a small subset
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)

In [11]:
for review, label in tst.take(1):
    print(review, label)
    print(imdb_encoder.decode(review))

tf.Tensor(
[71889 53896 87286 61224 31423 93816 83872 29236 82262 72896 93005 49907
 34944 93666 93861 64795 75364 62875 86463 42012 93720 87790 71889 69810
 65423 82262 87862 44540 77730 93005 89207 46454 87862 42012 73958 57972
 93316 31890 71889 93816 52772 88294 59375 71889 93816 56440 87286 38190
 89696 61627 77655 59597 60590 69640 42005 88620 71233 59160 87358 60590
 67883 75870 71233 73483 87862 33798 77114 51310 93150  2213 47463 83310
 91271 66853 73371 50067 78905 82644 83228 93666 53896 29343 87790 86451
 42005 81385 69563 93005 86451 93816 86166 53896 39105 55275 64408 78300
 92827 62363 83420 65896 86166 46272 86463 87360 84277 71889 69380 70593
 52772 84277 34944 93666 52772 79005 54279 62363 57972 80057 77095 50819
 49351     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
this was an

In [15]:
# now tokenize/encode/pad all training
# and testing data



encoded_train = imdb_train.map(encode_tf_fn,
                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn,
                             num_parallel_calls=tf.data.experimental.AUTOTUNE)

# GloVe Based Transfer Learning

## Loading Trained GloVe Embeddings

In [12]:
# Download the GloVe embeddings
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [13]:
#!unzip glove.6B.zip

In [16]:
# We will use the small 50D GLoVe vectors for this example
# It should be trivial to experiment with a different size

dict_w2v = {}
with open('../data/glove/glove.6B.50d.txt', encoding="utf8") as file:
    for line in file:
        tokens = line.split()
        word = tokens[0]
        vector = np.array(tokens[1:], dtype=np.float32)

        if vector.shape[0] == 50:
            dict_w2v[word] = vector
        else:
            print("There was an issue with " + word)

# lets check the vocabulary size
print("Dictionary Size: ", len(dict_w2v))

Dictionary Size:  400000


In [15]:
#import stanza
##en = stanza.download('en') 
#en = stanza.Pipeline(lang='en')
#dict_w2v = {}
#with open('../data/glove/glove.6B.50d.txt', encoding="utf8") as file:
#    for line in file:
#        tokenized = en(line)
#        for snt in tokenized.sentences:
#            for word in snt.tokens:
#                print(snt.tokens)
#                #vector = np.array(snt.tokens[1:], dtype=np.float32)
#                #dict_w2v[word] = vector
        

## lets check the vocabulary size
#print("Dictionary Size: ", len(dict_w2v))

## Building an Embedding Matrix
For each token in the training and test data, we will look up
the corresponding token in the GloVe dictionary loaded above.
Then, a mapping between the embeddings and token IDs will be 
created. This `embedding matrix` will be passed to the embedding
layer of the model as the pre-trained weights.

In [17]:
embedding_dim = 50
embedding_matrix = np.zeros((imdb_encoder.vocab_size, embedding_dim))

In [18]:
unk_cnt = 0
unk_set = set()
for word in imdb_encoder.tokens:
    embedding_vector = dict_w2v.get(word)

    if embedding_vector is not None:
        tkn_id = imdb_encoder.encode(word)[0]
        embedding_matrix[tkn_id] = embedding_vector
    else:
        unk_cnt += 1
        unk_set.add(word)

# Print how many werent found
print("Total unknown words: ", unk_cnt)

Total unknown words:  14553


## GloVe based BiLSTM Model

In [19]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=100

In [20]:
from tensorflow.keras.layers import Embedding, LSTM, \
                                    Bidirectional, Dense,\
                                    Dropout
            
def build_model_bilstm(vocab_size, embedding_dim, 
                       rnn_units, batch_size, train_emb=False):
  model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim, mask_zero=True,
              weights=[embedding_matrix], trainable=train_emb),
    #Dropout(0.25),  
    Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True, 
                                      dropout=0.5)),
    Bidirectional(tf.keras.layers.LSTM(rnn_units, dropout=0.25)),
    Dense(1, activation='sigmoid')
  ])
  return model

### Train a Feature Extraction Sequential Transfer Learning Model

In [21]:
model_fe = build_model_bilstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model_fe.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          4696550   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        58880     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 4,854,375
Trainable params: 157,825
Non-trainable params: 4,696,550
_________________________________________________________________


In [22]:
model_fe.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

In [23]:
# Prefetch for performance
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [24]:
print(encoded_train)
print(encoded_train_batched)

model_fe.fit(encoded_train_batched, epochs=10)

<ParallelMapDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>
<PrefetchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.int64)>
Epoch 1/10
250/250 [==============================] - 106s 265ms/step - loss: 0.6002 - accuracy: 0.6658 - precision: 0.6672 - recall: 0.66181s - loss: 0.6104 - accu
Epoch 2/10
250/250 [==============================] - 64s 257ms/step - loss: 0.5227 - accuracy: 0.7414 - precision: 0.7438 - recall: 0.7363
Epoch 3/10
250/250 [==============================] - 70s 280ms/step - loss: 0.4907 - accuracy: 0.7618 - precision: 0.7656 - recall: 0.7545
Epoch 4/10
250/250 [==============================] - 75s 299ms/step - loss: 0.4586 - accuracy: 0.7851 - precision: 0.7918 - recall: 0.7736
Epoch 5/10
250/250 [==============================] - 76s 304ms/step - loss: 0.4451 - accuracy: 0.7913 - precision: 0.7942 - recall: 0.786512s - loss: 0.4469 - accur
Epoch 6/10
250/250 [==============================] - 72s 287ms/step - loss: 0.4307 - accuracy: 0.

In [25]:
if tf.test.gpu_device_name(): 

    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Default GPU Device: /device:GPU:0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14054450616154680703
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4851040256
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17146640924310264810
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [26]:
model_fe.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 54s 168ms/step - loss: 0.3959 - accuracy: 0.8356 - precision: 0.8003 - recall: 0.894234s - loss: 0.4148 - accuracy: 0.8239 - precision: 0.7851 - recall - ETA - ETA: 28s - loss: 0.4111 - accuracy: 0.8278 - precision: 0.


[0.39589136838912964,
 0.8355600237846375,
 0.8003150224685669,
 0.8942400217056274]

In [ ]:
#model_fe.evaluate(["not very good","loved this"])

### Train a Fine-Tuning Sequential Transfer Learning Model

In [27]:
model_ft = build_model_bilstm(
  vocab_size=vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE,
  train_emb=True)

model_ft.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          4696550   
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 128)        58880     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 4,854,375
Trainable params: 4,854,375
Non-trainable params: 0
_________________________________________________________________


In [26]:
model_ft.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

In [27]:
model_ft.fit(encoded_train_batched, epochs=50)

Epoch 1/50
250/250 [==============================] - 77s 219ms/step - loss: 0.5549 - accuracy: 0.7056 - precision: 0.7088 - recall: 0.6980
Epoch 2/50
250/250 [==============================] - 59s 236ms/step - loss: 0.4033 - accuracy: 0.8192 - precision: 0.8183 - recall: 0.8205
Epoch 3/50
250/250 [==============================] - 58s 231ms/step - loss: 0.3294 - accuracy: 0.8590 - precision: 0.8599 - recall: 0.8578
Epoch 4/50
250/250 [==============================] - 43s 173ms/step - loss: 0.2735 - accuracy: 0.8863 - precision: 0.8843 - recall: 0.8888
Epoch 5/50
250/250 [==============================] - 45s 180ms/step - loss: 0.2377 - accuracy: 0.9045 - precision: 0.9056 - recall: 0.9031
Epoch 6/50
250/250 [==============================] - 44s 174ms/step - loss: 0.2082 - accuracy: 0.9170 - precision: 0.9174 - recall: 0.9165
Epoch 7/50
250/250 [==============================] - 43s 172ms/step - loss: 0.1783 - accuracy: 0.9318 - precision: 0.9344 - recall: 0.928910s - loss: 0
Epoch 8

In [29]:
model_ft.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 44s 147ms/step - loss: 1.1449 - accuracy: 0.8402 - precision: 0.8490 - recall: 0.827618s - loss: 1.1725 - accuracy: - ETA: 15s - loss: 1.1746 - accuracy: 0.8396 - precision: 0.8468 - recall:  - ETA: 15s - loss: 1.1764 - accuracy: 0.8393 - precis - ETA: 13s - loss: 1.17 - ETA: 7s - loss: 1.1596 - accuracy: 0.8388 - precision: 0 - ETA: 4s - loss: 1.1482 - accuracy: 0.8403 - preci


[1.1449368000030518,
 0.8402000069618225,
 0.8489946722984314,
 0.8276000022888184]

# BERT Based Transfer Learning With HuggingFace

In [16]:
# To clean up and free up GPU memory
# optional
tf.keras.backend.clear_session()

## Tokenization and Normalization with BERT

In [42]:

#model_ft.save('../data/model_ft')
#model_fe.save('../data/model_fe')

#!pip install transformers
#pip install transformers==3.0.2


os.environ.get("USE_TORCH", "AUTO").upper()

'AUTO'

In [17]:
from transformers import BertTokenizer

In [18]:
# Defining BERT tokenizer
#bert_name = 'bert-base-uncased'
bert_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(bert_name, add_special_tokens=True, 
                                          do_lower_case=False, max_length=150, 
                                          pad_to_max_length=True)

In [19]:
tst = "This was an absolutely terrible movie. Don't be lured in \
        by Christopher Walken or Michael Ironside."
tokenizer.encode_plus(tst, add_special_tokens=True, max_length=150, 
                      pad_to_max_length=True, 
                      return_attention_mask=True, 
                      return_token_type_ids=True,
                      truncation=True)

{'input_ids': [101, 1188, 1108, 1126, 7284, 6434, 2523, 119, 1790, 112, 189, 1129, 19615, 1181, 1107, 1118, 4978, 10065, 1424, 1137, 1847, 5621, 5570, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [20]:
tokens = tokenizer.encode(tst, add_special_tokens=True)
tokenizer.decode(tokens)

"[CLS] This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. [SEP]"

In [21]:
" ".join([tokenizer.decode([tok]) for tok in tokens])

"[CLS] This was an absolutely terrible movie . Don ' t be lure ##d in by Christopher Walk ##en or Michael Iron ##side . [SEP]"

In [22]:
tokenizer.encode_plus("Don't be lured", add_special_tokens=True, 
                      max_length=9,
                      pad_to_max_length=True, 
                      return_attention_mask=True, 
                      return_token_type_ids=True,
                      truncation=True)

{'input_ids': [101, 1790, 112, 189, 1129, 19615, 1181, 102, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0]}

In [23]:
tokenizer.encode_plus("Don't be"," lured", add_special_tokens=True, 
                      max_length=10,
                      pad_to_max_length=True, 
                      return_attention_mask=True, 
                      return_token_type_ids=True,
                      truncation=True
                     )

{'input_ids': [101, 1790, 112, 189, 1129, 102, 19615, 1181, 102, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]}

In [24]:
def bert_encoder(review):
    txt = review.numpy().decode('utf-8')
    encoded = tokenizer.encode_plus(txt, add_special_tokens=True, 
                                    max_length=150, pad_to_max_length=True, 
                                    return_attention_mask=True, 
                                    return_token_type_ids=True,
                                    truncation=True)
    return encoded['input_ids'], encoded['token_type_ids'], \
           encoded['attention_mask']


In [25]:
tst = imdb_train.take(5)

In [26]:
for review, label in tst:
    print(label)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)


In [27]:
bert_train = [bert_encoder(r) for r,l in imdb_train]
bert_lbl = [l for r, l in imdb_train]

In [49]:
bert_train = np.array(bert_train)
bert_train.shape

(25000, 3, 150)

In [28]:
bert_lbl = tf.keras.utils.to_categorical(bert_lbl, num_classes=2)
bert_lbl.shape

(25000, 2)

In [50]:
# create training and validation splits
from sklearn.model_selection import train_test_split

#print(bert_train.shape, bert_lbl.shape)


x_train, x_val, y_train, y_val = train_test_split(bert_train, bert_lbl, 
                                                    test_size=0.2, 
                                                    random_state=42)

print(x_train.shape, y_train.shape)

(20000, 3, 150) (20000, 2)


In [51]:
tr_reviews, tr_segments, tr_masks = np.split(x_train, 3, axis=1)
val_reviews, val_segments, val_masks = np.split(x_val, 3, axis=1)

In [52]:
tr_reviews.shape

(20000, 1, 150)

In [53]:
tr_reviews = tr_reviews.squeeze()
tr_segments = tr_segments.squeeze()
tr_masks = tr_masks.squeeze()

val_reviews = val_reviews.squeeze()
val_segments = val_segments.squeeze()
val_masks = val_masks.squeeze()

In [54]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y


train_ds = tf.data.Dataset.from_tensor_slices((tr_reviews, tr_masks, 
                                               tr_segments, y_train)).\
            map(example_to_features).shuffle(100).batch(16)

valid_ds = tf.data.Dataset.from_tensor_slices((val_reviews, val_masks, 
                                               val_segments, y_val)).\
            map(example_to_features).shuffle(100).batch(16)

## Pre-Built BERT Model for Classification

In [55]:
from transformers import TFBertForSequenceClassification

In [56]:
# Warnings can be ignored safely
bert_model = TFBertForSequenceClassification.from_pretrained(bert_name)

- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

bert_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [58]:
bert_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print("Fine-tuning BERT on IMDB")
#bert_model = tf.keras.models.load_model('../data/bertmodel')
bert_history = bert_model.fit(train_ds, epochs=2, validation_data=valid_ds)

Fine-tuning BERT on IMDB
Epoch 1/2
   1/1250 [..............................] - ETA: 6:31:26 - loss: 0.7625 - accuracy: 0.2500

In [63]:
bert_model.save('../data/bertmodel')

INFO:tensorflow:Assets written to: ../data/bertmodel\assets


INFO:tensorflow:Assets written to: ../data/bertmodel\assets


In [ ]:
new_model = tf.keras.models.load_model('../data/bertmodel')

In [60]:
# prep data for testing
bert_test = [bert_encoder(r) for r,l in imdb_test]
bert_tst_lbl = [l for r, l in imdb_test]

bert_test2 = np.array(bert_test)
bert_tst_lbl2 = tf.keras.utils.to_categorical(bert_tst_lbl, num_classes=2)

ts_reviews, ts_segments, ts_masks = np.split(bert_test2, 3, axis=1)
ts_reviews = ts_reviews.squeeze()
ts_segments = ts_segments.squeeze()
ts_masks = ts_masks.squeeze()

In [61]:
test_ds = tf.data.Dataset.from_tensor_slices((ts_reviews, ts_masks, 
                                               ts_segments, bert_tst_lbl2)).\
            map(example_to_features).shuffle(100).batch(16)

In [62]:
bert_model.evaluate(test_ds)

1563/1563 [==============================] - 447s 282ms/step - loss: 0.6933 - accuracy: 0.5096


[0.6932675838470459, 0.5096399784088135]

## Custom Model With BERT

In [66]:
from transformers import TFBertModel

In [67]:
bert_name = 'bert-base-cased'
bert = TFBertModel.from_pretrained(bert_name)

- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [68]:
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [69]:
max_seq_len = 150
inp_ids = tf.keras.layers.Input((max_seq_len,), dtype=tf.int64, name="input_ids")
att_mask = tf.keras.layers.Input((max_seq_len,), dtype=tf.int64, name="attention_mask")
seg_ids = tf.keras.layers.Input((max_seq_len,), dtype=tf.int64, name="token_type_ids")

In [70]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(None, 150), dtype=tf.int64, name=None),
  'attention_mask': TensorSpec(shape=(None, 150), dtype=tf.int64, name=None),
  'token_type_ids': TensorSpec(shape=(None, 150), dtype=tf.int64, name=None)},
 TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))

In [71]:
inp_dict = {"input_ids": inp_ids,
          "attention_mask": att_mask,
          "token_type_ids": seg_ids}
outputs = bert(inp_dict)

In [34]:
# lets see the output structure
outputs

(<tf.Tensor 'tf_bert_model/Identity:0' shape=(None, 150, 768) dtype=float32>,
 <tf.Tensor 'tf_bert_model/Identity_1:0' shape=(None, 768) dtype=float32>)

In [92]:
x = tf.keras.layers.Dropout(0.2)(outputs[1])
x = tf.keras.layers.Dense(200, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(2, activation='softmax')(x)

custom_model = tf.keras.models.Model(inputs=inp_dict, outputs=x)

In [91]:
train_ds.take(1)

<TakeDataset shapes: ({input_ids: (None, 150), attention_mask: (None, 150), token_type_ids: (None, 150)}, (None, 2)), types: ({input_ids: tf.int64, attention_mask: tf.int64, token_type_ids: tf.int64}, tf.float32)>

In [93]:
# first train the new layers added
bert.trainable = False  
optimizer = tf.keras.optimizers.Adam()  # standard learning rate
loss = tf.keras.losses.BinaryCrossentropy() #from_logits=True)
custom_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [94]:
print("Custom Model: training custom model on IMDB")
custom_history = custom_model.fit(train_ds, epochs=10, 
                                  validation_data=valid_ds)

Custom Model: training custom model on IMDB
Epoch 1/10
1250/1250 [==============================] - 205s 164ms/step - loss: 0.2915 - accuracy: 0.9096 - val_loss: 0.3498 - val_accuracy: 0.8614
Epoch 2/10
1250/1250 [==============================] - 207s 165ms/step - loss: 0.2664 - accuracy: 0.9118 - val_loss: 0.3537 - val_accuracy: 0.8618
Epoch 3/10
1250/1250 [==============================] - 211s 168ms/step - loss: 0.2627 - accuracy: 0.9128 - val_loss: 0.3464 - val_accuracy: 0.8616
Epoch 4/10
1250/1250 [==============================] - 211s 169ms/step - loss: 0.2597 - accuracy: 0.9137 - val_loss: 0.3386 - val_accuracy: 0.8616
Epoch 5/10
1250/1250 [==============================] - 211s 169ms/step - loss: 0.2589 - accuracy: 0.9122 - val_loss: 0.3814 - val_accuracy: 0.8616
Epoch 6/10
1250/1250 [==============================] - 211s 169ms/step - loss: 0.2601 - accuracy: 0.9139 - val_loss: 0.3322 - val_accuracy: 0.8624
Epoch 7/10
1250/1250 [==============================] - 211s 169ms/s

In [95]:
custom_model.evaluate(test_ds)

1563/1563 [==============================] - 193s 123ms/step - loss: 0.3239 - accuracy: 0.8667


[0.3239259719848633, 0.8666800260543823]

In [96]:
# Now finetune BERT for a couple of epochs
bert.trainable = True
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy() #from_logits=True)

custom_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [97]:
custom_model.summary()

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
attention_mask (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_ids (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 150, 768), ( 108310272   attention_mask[0][0]             
                                                                 input_ids[0][0]      

In [99]:
print("Custom Model: Fine-tuning BERT on IMDB")
custom_history = custom_model.fit(train_ds, epochs=2, 
                                  validation_data=valid_ds)

Custom Model: Fine-tuning BERT on IMDB
Epoch 1/2
1250/1250 [==============================] - 470s 376ms/step - loss: 0.0845 - accuracy: 0.9754 - val_loss: 0.4150 - val_accuracy: 0.8874
Epoch 2/2
1250/1250 [==============================] - 470s 376ms/step - loss: 0.0494 - accuracy: 0.9862 - val_loss: 0.4479 - val_accuracy: 0.8870


In [100]:
custom_model.evaluate(test_ds)

1563/1563 [==============================] - 196s 126ms/step - loss: 0.5060 - accuracy: 0.8750


[0.5059850215911865, 0.8749600052833557]